In [2]:
import os

import numpy as np
import math
import random

import torch

import torch.nn as nn 
from torch.nn.utils import parameters_to_vector
import torch.optim as optim
from torchinfo import summary

from brevitas.export import export_onnx_qcdq

import config_aimet
import models_aimet_medium_fasdd
import models
import utils
import datasets
import metrics
import loss
import val_epoch

# Load Model

In [3]:
if config_aimet.MODEL == "BED":
    
    print("Using Fixed Point Quantizers without BN")
    quant_model = models_aimet_medium_fasdd.QUANT_MEDIUM_PRUNING_AFTER_SVD_CLASSIFIER(
            weight_bw = config_aimet.WEIGHTS_BIT_WIDTH,
            big_layers_weight_bw = config_aimet.BIG_LAYERS_WEIGHTS_BIT_WIDTH,
            act_bw = config_aimet.ACTIVATIONS_BIT_WIDTH,
            bias_bw = config_aimet.BIAS_BIT_WIDTH,
            num_classes=config_aimet.N_CLASSES).to(config_aimet.DEVICE)
else:
    print("Wrong Model")
    raise SystemExit("Wrong Model")

# MODEL PARAMETERS
n_trainable = sum(p.numel() for p in quant_model.parameters() if p.requires_grad)
print(f'\nTrainable parameters = {n_trainable}')

n_params = parameters_to_vector(quant_model.parameters()).numel()
print(f'Total parameters = {n_params}\n')

Using Fixed Point Quantizers without BN

Trainable parameters = 63631
Total parameters = 63631



### FASDD VAL added: Model with Medium Compression: conv341 defined as big layer

In [4]:
model_dir = 'experiments_fuseBN_256_fasdd/test_v30_FASDD_VAL__MED_w4W3a8b4_FxdPnt_MSE_PerChnlW_IntBiasIntScl/weights/'
model_file = model_dir + 'BED_classifier__best_mean_F1.pt'
epochs_trained = utils.load_checkpoint(
    model_file, 
    quant_model, 
    optimizer=None, 
    scheduler=None, 
    device=config_aimet.DEVICE)

Loading Model. Trained during 94 epochs


### FASDD VAL added: Model with No Compression

In [5]:
# model_dir = 'experiments_fuseBN_256_fasdd/test_v21_FASDD_VAL__NoCOMP_w4W2a8b4_FxdPnt_MSE_PerChnlW_IntBiasIntScl/weights/'
# model_file = model_dir + 'BED_classifier__best_mean_F1.pt'
# epochs_trained = utils.load_checkpoint(model_file, quant_model, optimizer=None, scheduler=None, device=config_aimet.DEVICE)

### Model to Eval Mode

In [6]:
quant_model.eval()
print("Model to eval mode")

Model to eval mode


## Remove Quant Identity Layer from the Start

In [7]:
class QUANT_MEDIUM_PRUNING_AFTER_SVD_CLASSIFIER_NO_QuantIdentity(nn.Module):
    def __init__(self, base_model):
        super(QUANT_MEDIUM_PRUNING_AFTER_SVD_CLASSIFIER_NO_QuantIdentity, self).__init__()
        self.model = nn.Sequential(*list(base_model.children())[0][1:])

    def forward(self, x):
        x = self.model(x)
        return x

In [8]:
no_identity_model = QUANT_MEDIUM_PRUNING_AFTER_SVD_CLASSIFIER_NO_QuantIdentity(quant_model)

In [9]:
no_identity_model.eval()
print("Model to eval mode")

Model to eval mode


# Compare Weights and Bias

In [10]:
torch.allclose(quant_model.model.conv1.bias, no_identity_model.model[0].bias)
#print(quant_model.model.conv1.weight)

True

# Evaluate Brevitas Quant Model

## Dataset

In [11]:
val_loader = datasets.get_val_loader()


TEST DFire dataset
DFire Removed wrong images: 0
DFire empty images: 2005
DFire only smoke images: 1186
DFire only fire images: 220
DFire smoke and fire images: 895

Test dataset len: 4306

TEST FASDD UAV dataset
FASDD Removed wrong images: 0
FASDD empty images: 1997
FASDD only smoke images: 846
FASDD only fire images: 35
FASDD smoke and fire images: 1303

Test FASDD UAV dataset len: 4181

TEST FASDD CV dataset
FASDD Removed wrong images: 0
FASDD empty images: 6533
FASDD only smoke images: 3902
FASDD only fire images: 2091
FASDD smoke and fire images: 3358

Test FASDD CV dataset len: 15884
Concatenate Test DFire and FASDD UAV datasets
Test dataset len: 8487
Concatenate with FASDD CV dataset
Test dataset len: 24371


## Loss
Needed for evaluation function

In [12]:
if config_aimet.LOSS_FN == "BCE":
    print(f'Loss Function: BCE')
    print(f'Smoke Precision Weight: {config_aimet.SMOKE_PRECISION_WEIGHT}')
    loss_fn = loss.BCE_LOSS(device=config_aimet.DEVICE, smoke_precision_weight=config_aimet.SMOKE_PRECISION_WEIGHT)
else:
    print("Wrong loss function")
    raise SystemExit("Wrong loss function")

Loss Function: BCE
Smoke Precision Weight: 0.8


## Quant Model Evaluation

In [13]:
quant_model.eval()
with torch.no_grad():
    val_losses, val_metrics = val_epoch.eval_fn(
        loader=val_loader, 
        model=quant_model,                         
        loss_fn=loss_fn,
        device=config_aimet.DEVICE)

Validating:   0%|                                                                                                                                                    | 0/380 [00:00<?, ?it/s]/opt/conda/envs/pytorch_brevitas/lib/python3.10/site-packages/torch/_tensor.py:1362: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at /opt/conda/conda-bld/pytorch_1702400410390/work/c10/core/TensorImpl.h:1900.)
  return super().rename(names)
Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 380/380 [00:34<00:00, 11.12it/s]

Total Loss  |Smoke Loss  |Fire Loss   
------------ ------------ ------------
16.603      |10.941      |5.662       
SMOKE -> Precision: 0.955 - Recall: 0.885 - Accuracy: 0.926 - F1: 0.919
FIRE -> Precision: 0.930 - Recall: 0.971 - Accuracy: 0.967 - F1: 0.950


In [14]:
for k, v in val_metrics.items():
    print(f'{k}: smoke {v[0]:.4f} - fire: {v[1]:.4f}')
print(f'F1 Mean: {(val_metrics["F1"][0] + val_metrics["F1"][1])/2:.4f}')

Accuracy: smoke 0.9263 - fire: 0.9669
Precision: smoke 0.9551 - fire: 0.9302
Recall: smoke 0.8853 - fire: 0.9707
F1: smoke 0.9189 - fire: 0.9500
F1 Mean: 0.9345


## No Identity Model Evaluation

In [15]:
no_identity_model.eval()
with torch.no_grad():
    val_losses, val_metrics = val_epoch.eval_fn(
        loader=val_loader, 
        model=no_identity_model,                         
        loss_fn=loss_fn,
        device=config_aimet.DEVICE)

Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 380/380 [00:33<00:00, 11.19it/s]

Total Loss  |Smoke Loss  |Fire Loss   
------------ ------------ ------------
16.603      |10.941      |5.662       
SMOKE -> Precision: 0.955 - Recall: 0.885 - Accuracy: 0.926 - F1: 0.919
FIRE -> Precision: 0.930 - Recall: 0.971 - Accuracy: 0.967 - F1: 0.950


In [16]:
for k, v in val_metrics.items():
    print(f'{k}: smoke {v[0]:.4f} - fire: {v[1]:.4f}')
print(f'F1 Mean: {(val_metrics["F1"][0] + val_metrics["F1"][1])/2:.4f}')

Accuracy: smoke 0.9263 - fire: 0.9669
Precision: smoke 0.9551 - fire: 0.9302
Recall: smoke 0.8853 - fire: 0.9707
F1: smoke 0.9189 - fire: 0.9500
F1 Mean: 0.9345


# Model to CPU and ONNX Export

In [17]:
# quant_model.to('cpu')
# print("Model to CPU before exports")

In [18]:
# export_onnx_qcdq(
#     quant_model, 
#     torch.randn(1, 3, config_aimet.IMG_H, config_aimet.IMG_W).to('cpu'), 
#     export_path='./models/onnx_export/medium_fassd__conv341_big__epoch=93.onnx')

# Export to QONNX, previous step to FINN-ONNX

In [19]:
from brevitas.export import export_qonnx

In [19]:
no_identity_model.cpu()
print("Moved to CPU")

Moved to CPU


### Export Function

In [20]:
export_qonnx(
    no_identity_model, 
    export_path='./experiments_fuseBN_256_fasdd/test_v30_FASDD_VAL__MED_w4W3a8b4_FxdPnt_MSE_PerChnlW_IntBiasIntScl/BED_classifier__med_comp__NO_Identity__FLOOR__QONNX_OPSET_9.onnx',
    input_t=torch.randn(1, 3, config_aimet.IMG_H, config_aimet.IMG_W).to('cpu') 
)

ir_version: 8
producer_name: "pytorch"
producer_version: "2.1.2"
graph {
  node {
    input: "/model/model.0/weight_quant/export_handler/Constant_1_output_0"
    input: "/model/model.0/weight_quant/export_handler/Constant_2_output_0"
    input: "/model/model.0/weight_quant/export_handler/Constant_3_output_0"
    input: "/model/model.0/weight_quant/export_handler/Constant_output_0"
    output: "/model/model.0/weight_quant/export_handler/Quant_output_0"
    name: "/model/model.0/weight_quant/export_handler/Quant"
    op_type: "Quant"
    attribute {
      name: "narrow"
      i: 1
      type: INT
    }
    attribute {
      name: "rounding_mode"
      s: "ROUND"
      type: STRING
    }
    attribute {
      name: "signed"
      i: 1
      type: INT
    }
    domain: "onnx.brevitas"
  }
  node {
    input: "model.0.bias"
    input: "/model/model.0/bias_quant/export_handler/Constant_output_0"
    input: "/model/model.0/weight_quant/export_handler/Constant_3_output_0"
    input: "/model/mo

# New Model with BIPOLAR output

In [21]:
import brevitas.nn as qnn

In [22]:
class QUANT_MEDIUM_PRUNING_AFTER_SVD_CLASSIFIER_NO_QuantIdentity__BIPOLAR_Out(nn.Module):
    def __init__(self, base_model):
        super(QUANT_MEDIUM_PRUNING_AFTER_SVD_CLASSIFIER_NO_QuantIdentity__BIPOLAR_Out, self).__init__()
        self.model = nn.Sequential(*list(base_model.children())[0][1:])
        self.qnt_output = qnn.QuantIdentity(
            quant_type='binary', 
            scaling_impl_type='const',
            bit_width=1, min_val=-1.0, max_val=1.0)

    def forward(self, x):
        x = self.model(x)
        x = self.qnt_output(x)
        return x

In [23]:
no_identity_bipolar_out_model = QUANT_MEDIUM_PRUNING_AFTER_SVD_CLASSIFIER_NO_QuantIdentity__BIPOLAR_Out(quant_model).to(config_aimet.DEVICE)

## New Val Function to test BIPOLAR Out

In [24]:
from tqdm import tqdm

def eval_bipolar_fn(loader, model, device):
    
    model.eval()
    loop = tqdm(loader, desc='Validating', leave=True)

    for batch_idx, (x, y) in enumerate(loop):
        x, y = x.to(device), y.to(device)
        yhat = model(x)

        yhat[yhat < 1] = 0
    
        metrics.precision_metric.update(yhat, y)
        metrics.recall_metric.update(yhat, y)
        metrics.accuracy_metric.update(yhat, y)
        metrics.f1_metric.update(yhat, y)
   
    precision = metrics.precision_metric.compute()
    recall = metrics.recall_metric.compute()
    accuracy = metrics.accuracy_metric.compute()
    f1 = metrics.f1_metric.compute()
    
    metrics.precision_metric.reset()
    metrics.recall_metric.reset()
    metrics.accuracy_metric.reset()
    metrics.f1_metric.reset()

    print(f'SMOKE -> Precision: {precision[0]:.4f} - Recall: {recall[0]:.4f} - Accuracy: {accuracy[0]:.4f} - F1: {f1[0]:.4f}')
    print(f'FIRE -> Precision: {precision[1]:.4f} - Recall: {recall[1]:.4f} - Accuracy: {accuracy[1]:.4f} - F1: {f1[1]:.4f}')
    
    return (
        {
        'Accuracy': [accuracy[0].item(), accuracy[1].item()],
        'Precision': [precision[0].item(), precision[1].item()],
        'Recall': [recall[0].item(), recall[1].item()],
        'F1': [f1[0].item(), f1[1].item()] 
        }
    )

In [25]:
no_identity_bipolar_out_model.eval()
with torch.no_grad():
    val_metrics = eval_bipolar_fn(
        loader=val_loader, 
        model=no_identity_bipolar_out_model,                         
        device=config_aimet.DEVICE)

Validating: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 380/380 [00:34<00:00, 11.12it/s]

SMOKE -> Precision: 0.9550 - Recall: 0.8857 - Accuracy: 0.9264 - F1: 0.9190
FIRE -> Precision: 0.9302 - Recall: 0.9707 - Accuracy: 0.9669 - F1: 0.9500


In [26]:
for k, v in val_metrics.items():
    print(f'{k}: smoke {v[0]:.4f} - fire: {v[1]:.4f}')
print(f'F1 Mean: {(val_metrics["F1"][0] + val_metrics["F1"][1])/2:.4f}')

Accuracy: smoke 0.9264 - fire: 0.9669
Precision: smoke 0.9550 - fire: 0.9302
Recall: smoke 0.8857 - fire: 0.9707
F1: smoke 0.9190 - fire: 0.9500
F1 Mean: 0.9345


# Export to QONNX

In [27]:
no_identity_bipolar_out_model.cpu()
print("Moved to CPU")

Moved to CPU


### Export Function

In [28]:
export_qonnx(
    no_identity_bipolar_out_model, 
    export_path='./experiments_fuseBN_256_fasdd/test_v30_FASDD_VAL__MED_w4W3a8b4_FxdPnt_MSE_PerChnlW_IntBiasIntScl/BED_classifier__med_comp__NO_Identity__BIPOLAR__FLOOR__QONNX.onnx',
    input_t=torch.randn(1, 3, config_aimet.IMG_H, config_aimet.IMG_W).to('cpu') 
)

ir_version: 8
producer_name: "pytorch"
producer_version: "2.1.2"
graph {
  node {
    input: "/model/model.0/weight_quant/export_handler/Constant_1_output_0"
    input: "/model/model.0/weight_quant/export_handler/Constant_2_output_0"
    input: "/model/model.0/weight_quant/export_handler/Constant_3_output_0"
    input: "/model/model.0/weight_quant/export_handler/Constant_output_0"
    output: "/model/model.0/weight_quant/export_handler/Quant_output_0"
    name: "/model/model.0/weight_quant/export_handler/Quant"
    op_type: "Quant"
    attribute {
      name: "narrow"
      i: 1
      type: INT
    }
    attribute {
      name: "rounding_mode"
      s: "ROUND"
      type: STRING
    }
    attribute {
      name: "signed"
      i: 1
      type: INT
    }
    domain: "onnx.brevitas"
  }
  node {
    input: "model.0.bias"
    input: "/model/model.0/bias_quant/export_handler/Constant_output_0"
    input: "/model/model.0/weight_quant/export_handler/Constant_3_output_0"
    input: "/model/mo